In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import pickle

with open('2_img_test.pkl', 'rb') as f:
    ds = pickle.load(f)


# add module to path so python can import it. This is the directory with
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/spicy_snow')

# import the function to test
from processing.s1_preprocessing import s1_amp_to_dB


# plot first time slice and VV band to see initial
ds['s1'].isel(time=0).sel(band='VV').plot()
plt.show()

# snag the original time slice to compare afterwards
amp = ds['s1'].isel(time=0).sel(band='VV').values.ravel()
amp = amp[amp != 0]

# run your newly written function over the dataset and capture returning dataset
s1_amp_to_dB(ds)

# plot first time slice and VV band to see dB change

ds['s1'].isel(time=0).sel(band='VV').plot()
plt.show()

# grab the first time slice VV
dB = ds['s1'].isel(time=0).sel(band='VV').values.ravel()
dB = dB[np.isfinite(dB)]

# assert the conversion was succesful
assert np.allclose(dB, 10 * np.log10(amp))

In [ ]:
def s1_clip_outliers(dataset: xr.Dataset, inplace: bool=False) -> xr.Dataset:
    """
    Remove s1 image outliers by masking pixels 3 dB above 90th percentile or
    3 dB before the 10th percentile. (-35 -> 15 dB for VV) and (-40 -> 10 for VH
    in HL's code - line # 291)

    Args:
    dataset: Xarray Dataset of sentinel images to clip outliers
    inplace: boolean flag to modify original Dataset or return a new Dataset

    Returns:
    dataset: Xarray Dataset of sentinel images with masked outliers
    """
    # Check inplace flag
    if inplace:
        ds = dataset
        rtrn = False
    else:
        ds = dataset.copy(deep=True)
        rtrn = True

    # Calculate time series 10th and 90th percentile 
    # Threshold vals 3 dB above/below percentiles
    for band in ['VV','VH']:
        data = ds['s1'].sel(band=band)
        vals = ds['s1'].sel(band=band).values.flatten()
        vals_valid = vals[~np.isnan(vals)]
        thresh_lo = np.percentile(vals_valid, 10) - 3
        thresh_hi = np.percentile(vals_valid, 90) + 3
        # Mask using percentile thresholds
        data_masked = data.where((data > thresh_lo) & (data < thresh_hi))
        ds['s1'].loc[dict(band = band)] = data_masked

    if rtrn:
        return ds


In [ ]:
test = s1_clip_outliers(ds, inplace=False)

for band in ['VV','VH']:
    print(f'{band} nans before:', sum(np.isnan(ds['s1'].sel(band=band).data.flatten())))
    print(f'{band} nans after:', sum(np.isnan(test['s1'].sel(band=band).data.flatten())))


In [ ]:
ds_copy = ds

ds_copy['s1'].loc[dict(band = 'VV')] = ds['s1'].sel(band='VV').where((ds['s1'].sel(band='VV') > -7.7) & (ds['s1'].sel(band='VV') < -6.8))
# ds_copy['s1'].loc[dict(band = 'VV')].data.shape
# ds['s1'].sel(band='VV').where((ds['s1'].sel(band='VV') > -7.7) & (ds['s1'].sel(band='VV') < -6.8)).data.shape
# ds_copy['s1']
ds['s1']


In [ ]:
test_ds = xr.Dataset(data_vars={'ims':ds['ims'], 'fcf':ds['fcf']})
test_ds.assign(s1=test)
test_ds

In [ ]:
ds['s1'].band